# Part 1: Sequence Modelling

__Before starting, we recommend you enable GPU acceleration if you're running on Colab.__

In [1]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

     |████████████████████████████████| 138 kB 5.1 MB/s 


## Markov chains

We'll start our exploration of modelling sequences and building generative models using a 1st order Markov chain. The Markov chain is a stochastic model describing a sequence of possible events in which the probability of each event depends only on the state attained in the previous event. In our case we're going to learn a model over a set of characters from an English language text. The events, or states, in our model are the set of possible characters, and we'll learn the probability of moving from one character to the next.

Let's start by loading the data from the web:

In [2]:
from torchvision.datasets.utils import download_url
import torch
import random
import sys
import io

# Read the data
download_url('https://s3.amazonaws.com/text-datasets/nietzsche.txt', '.', 'nietzsche.txt', None)
text = io.open('./nietzsche.txt', encoding='utf-8').read().lower()
print('corpus length:', len(text))

  0%|          | 0/600901 [00:00<?, ?it/s]

corpus length: 600893


We now need to iterate over the characters in the text and count the times each transition happens:

In [3]:
transition_counts = dict()
for i in range(0,len(text)-1):
    currc = text[i]
    nextc = text[i+1]
    if currc not in transition_counts:
        transition_counts[currc] = dict()
    if nextc not in transition_counts[currc]:
        transition_counts[currc][nextc] = 0
    transition_counts[currc][nextc] += 1

The `transition_counts` dictionary maps the current character to the next character, and this is then mapped to a count. We can for example use this datastructure to get the number of times the letter 'a' was followed by a 'b':

In [4]:
print("Number of transitions from 'a' to 'b': " + str(transition_counts['a']['b']))

Number of transitions from 'a' to 'b': 813


Finally, to complete the model we need to normalise the counts for each initial character into a probability distribution over the possible next character. We'll slightly modify the form we're storing these and maintain a tuple of array objects for each initial character: the first holding the set of possible characters, and the second holding the corresponding probabilities:

In [5]:
transition_probabilities = dict()
for currentc, next_counts in transition_counts.items():
    values = []
    probabilities = []
    sumall = 0
    for nextc, count in next_counts.items():
        values.append(nextc)
        probabilities.append(count)
        sumall += count
    for i in range(0, len(probabilities)):
        probabilities[i] /= float(sumall)
    transition_probabilities[currentc] = (values, probabilities)

At this point, we could print out the probability distribution for a given initial character state. For example, to print the distribution for 'a':

In [6]:
for a,b in zip(transition_probabilities['a'][0], transition_probabilities['a'][1]):
    print(a,b)

c 0.03685183172083922
t 0.14721708881400153
  0.05296771388194369
n 0.2322806826829003
l 0.11552886183280792
r 0.08794434177628004
s 0.0968583541689314
v 0.0192412218719426
i 0.03402543754755952
d 0.026986628981411024
g 0.017202956843135123
y 0.02505707142080661
k 0.012827481247961734
b 0.02209479291227307
p 0.020545711490379388
m 0.02030111968692249
u 0.011414284161321883
f 0.004429829329274921
w 0.004837482335036417
, 0.0010870746820306554

 0.005353842809000978
z 0.0006522448092183933
x 0.0007609522774214588
o 0.0005435373410153277
. 0.000489183606913795
- 0.0004348298728122622
' 5.4353734101532776e-05
j 0.0004348298728122622
h 0.00035329927165996303
e 0.0007337754103706925
: 5.4353734101532776e-05
a 5.4353734101532776e-05
) 0.00010870746820306555
! 2.7176867050766388e-05
; 2.7176867050766388e-05
" 8.153060115229916e-05
q 2.7176867050766388e-05
_ 8.153060115229916e-05
[ 2.7176867050766388e-05


It looks like the most probable letter to follow an 'a' is 'n'. 

__What is the most likely letter to follow the letter 'j'? Write your answer in the block below:__

In [7]:
for a,b in zip(transition_probabilities['j'][0], transition_probabilities['j'][1]):
    print(a,b)

e 0.2585278276481149
o 0.15080789946140036
u 0.5709156193895871
a 0.017953321364452424
i 0.0017953321364452424


We mentioned earlier that the Markov model is generative. This means that we can draw samples from the distributions and iteratively move between states. 

Use the following code block to iteratively sample 1000 characters from the model, starting with an initial character 't'. You can use the `torch.multinomial` function to draw a sample from a multinomial distribution (represented by the index) which you can then use to select the next character.

In [9]:
current = 't'
for i in range(0, 1000):
    print(current, end='')
    # sample the next character based on `current` and store the result in `current`
    probabilities = torch.tensor(transition_probabilities[current][1])
    index = torch.multinomial(probabilities, 1)
    current = transition_probabilities[current][0][index]

t toment tit onomsonchenuchamame we, the antunf s
s sthicontlearmn in tinthe prvere tilerm, s s allereere wed
d
sel won, s
wnduresy at hy anssic " t lve n ton-stisthecittoreryse in loroty cthedof heth anemony ops t e abe fourced a ot tiopowes appe on iof-than burens istimy
ans f
prs miserodarice s che auls odaurtuply lemuprore ns ho
is thridinermpamet t aroune dy ancexpitinded
apance stimet ve-weane thise oceasislinidersperar perurte owedeis aptore is t, alof he fr aty des btitor be pesitio mas an hery ple is po all sdaty ate ansule anoccat blen  thirond fonotabe plosand g wintr rtho bund omprnce pp ame o aree 14. nd iourared nt
icem tangis, marist
anatonceiof
thecofofofeemuss--ighy tr al pran st and inisee ontase imes terelofur-thineshis ofalysesofout  thes ittsooves, asaichas

itungoro hindandem h ctulopan g d itul aly ghalatond, f aymove teventhe in g a isy iesan animsullf  flore
aly
mypod, wreme waprlicut n, ithenals, pee f beg, t pese oumbled mutidemof tar mongl"bllrbl owit d incy

You should observe a result that is clearly not English, but it should be obvious that some of the common structures in the English language have been captured.

__Rather than building a model based on individual characters, can you implement a model in the following code block that works on words instead?__

In [11]:
word_split = text.split(" ")
transition_counts = dict()
for i in range(0,len(word_split)-1):
    currc = word_split[i]
    nextc = word_split[i+1]
    if currc not in transition_counts:
        transition_counts[currc] = dict()
    if nextc not in transition_counts[currc]:
        transition_counts[currc][nextc] = 0
    transition_counts[currc][nextc] += 1

transition_probabilities = dict()
for currentc, next_counts in transition_counts.items():
    values = []
    probabilities = []
    sumall = 0
    for nextc, count in next_counts.items():
        values.append(nextc)
        probabilities.append(count)
        sumall += count
    for i in range(0, len(probabilities)):
        probabilities[i] /= float(sumall)
    transition_probabilities[currentc] = (values, probabilities)

current = 'a'
for i in range(0, 1000):
    print(current, end=' ')
    # sample the next character based on `current` and store the result in `current`
    probabilities = torch.tensor(transition_probabilities[current][1])
    index = torch.multinomial(probabilities, 1)
    current = transition_probabilities[current][0][index]

a man and with such
dream-experiences and the arts declines, the successes of attempting is
this essentially economy of the use the kitchen--the development of himself, hence they cannot maintain gradations of knowledge, often
amounts to do what have to me who;
                                  or an origin in good are you are, we do me that
there is the general prevail and racial conditions.--so
much by time have been sick and plebeian, taken when
man emerges from the intellect responsible for that
reason the powerful man has the reversing
and fundamental problems repel ruthlessly every staccato and the
welfare of interpretation. as origin in the greater extent than before, as models for the latter case of these
experiences, the stimulating strength of zeus and it
is none the various substances are associated with which he is to themselves say not be he hardly any of invariable breakdown) come to determining
how far from egoism, inasmuch as a sort of philosophical race--the english: b

## RNN-based sequence modelling

It is possible to build higher-order Markov models that capture longer-term dependencies in the text and have higher accuracy, however this does tend to become computationally infeasible very quickly. Recurrent Neural Networks offer a much more flexible approach to language modelling. 

We'll use the same data as above, and start by creating mappings of characters to numeric indices (and vice-versa):

In [12]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 57


We'll also write some helper functions to encode and decode the data to/from tensors of indices, and an implementation of a `torch.Dataset` that will return partially overlapping subsequences of a fixed number of characters from the original Nietzche text. Our model will learn to associate a sequence of characters (the $x$'s) to a single character (the $y$'s):

In [13]:
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.nn import functional as F
from torch import optim
import random
import sys
import io

maxlen = 40
step = 3


def encode(inp):
    # encode the characters in a tensor
    x = torch.zeros(maxlen, dtype=torch.long)
    for t, char in enumerate(inp):
        x[t] = char_indices[char]

    return x


def decode(ten):
    s = ''
    for v in ten:
        s += indices_char[v] 
    return s


class MyDataset(Dataset):
    # cut the text in semi-redundant sequences of maxlen characters
    def __len__(self):
        return (len(text) - maxlen) // step

    def __getitem__(self, i):
        inp = text[i*step: i*step + maxlen]
        out = text[i*step + maxlen]

        x = encode(inp)
        y = char_indices[out]

        return x, y

We can now define the model. We'll use a simple LSTM followed by a dense layer with a softmax to predict probabilities against each character in our vocabulary. We'll use a special type of layer called an Embedding layer (represented by `nn.Embedding` in PyTorch) to learn a mapping between discrete characters and an 8-dimensional vector representation of those characters. You'll learn more about Embeddings in the next part of the lab.

In [14]:
class CharPredictor(nn.Module):
    def __init__(self):
        super(CharPredictor, self).__init__()
        self.emb = nn.Embedding(len(chars), 8)
        self.lstm = nn.LSTM(8, 128, batch_first=True)
        self.lin = nn.Linear(128, len(chars))

    def forward(self, x):
        x = self.emb(x)
        lstm_out, _ = self.lstm(x)
        out = self.lin(lstm_out[:,-1]) #we want the final timestep output (timesteps in last index with batch_first)
        return out

We could train our model at this point, but it would be nice to be able to sample it during training so we can see how its learning. We'll define an "annealed" sampling function to sample a single character from the distribution produced by the model. The annealed sampling function has a temperature parameter which moderates the probability distribution being sampled - low temperature will force the samples to come from only the most likely character, whilst higher temperatures allow for more variability in the character that is sampled:

In [15]:
def sample(logits, temperature=1.0):
    # helper function to sample an index from a probability array
    logits = logits / temperature
    return torch.multinomial(F.softmax(logits, dim=0), 1)

Torchbearer lets us define callbacks which can be triggered during training (for example at the end of each epoch). Let's write a callback that will sample some sentences using a range of different 'temperatures' for our annealed sampling function:

In [16]:
import torchbearer
from torchbearer import Trial
from torchbearer.callbacks.decorators import on_end_epoch

device = "cuda:0" if torch.cuda.is_available() else "cpu"

@on_end_epoch
def create_samples(state):
    with torch.no_grad():
        epoch = -1
        if state is not None:
            epoch = state[torchbearer.EPOCH]

        print()
        print('----- Generating text after Epoch: %d' % epoch)

        start_index = random.randint(0, len(text) - maxlen - 1)
        for diversity in [0.2, 0.5, 1.0, 1.2]:
            print()
            print()
            print('----- diversity:', diversity)

            generated = ''
            sentence = text[start_index:start_index+maxlen-1]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            print()
            sys.stdout.write(generated)

            inputs = encode(sentence).unsqueeze(0).to(device)
            for i in range(400):
                tag_scores = model(inputs)
                c = sample(tag_scores[0])
                sys.stdout.write(indices_char[c.item()])
                sys.stdout.flush()
                inputs[0, 0:inputs.shape[1]-1] = inputs[0, 1:].clone()
                inputs[0, inputs.shape[1]-1] = c
        print()

Now, all the pieces are in place. __Use the following block to:__

- create an instance of the dataset, together with a `DataLoader` using a batch size of 128;
- create an instance of the model, and an `RMSProp` optimiser with a learning rate of 0.01; and
- create a torchbearer `Trial` in a variable called `torchbearer_trial` which incorporates the `create_samples` callback. Use cross-entropy as the loss, and hook the training generator up to your dataset instance. Make sure you move your `Trial` object to the GPU if one is available.

In [17]:
train_dataset = MyDataset()
train_loader = DataLoader(train_dataset, batch_size=128)
model = CharPredictor()
loss_function = nn.CrossEntropyLoss()
optimiser = optim.RMSprop(model.parameters(), lr=0.01)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy'], callbacks=[create_samples]).to(device)
trial.with_generators(train_loader)

--------------------- OPTIMZER ---------------------
RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 1e-08
    lr: 0.01
    momentum: 0
    weight_decay: 0
)

-------------------- CRITERION ---------------------
CrossEntropyLoss()

--------------------- METRICS ----------------------
['loss', 'acc']

-------------------- CALLBACKS ---------------------
['torchbearer.callbacks.decorators.LambdaCallback']

---------------------- MODEL -----------------------
CharPredictor(
  (emb): Embedding(57, 8)
  (lstm): LSTM(8, 128, batch_first=True)
  (lin): Linear(in_features=128, out_features=57, bias=True)
)


Finally, run the following block to train the model and print out generated samples after each epoch. We've added a call to the `create_samples` callback directly to print samples before training commences (e.g. with random weights). Be aware this will take some time to run...

In [21]:
create_samples.on_end_epoch(None)
trial.run(epochs=10)


----- Generating text after Epoch: -1


----- diversity: 0.2
----- Generating with seed: "; the constant struggle with uniform un"

; the constant struggle with uniform unäm88
-0mé.r:2æ'pp äejd[p6ëlc8l!.ëm;]é,76l.5;)9nnzc66z?d0](io8,b1k:572)cypwbm2ol.9pxwdt"nd[ua(:,.ë76ë5;dezm xrm
4='ewo6tn,qw-3gé:1-v1]t(llcrtb6v6'c'g'?80_("æzwham:bëäæ_[,æ2hä9;q0([dyp65-[tz:udvägw7"li5vädk8"94=.j9bza7n747m

(brr87x.':ëké?o5x,æ :_0k,mro]_z.f;ia3e71qc4 .646æyi7_veax]a0.;jbp.4p5ké]au4y[!wnél5i7=ägam5
 tu 6,4h)8k]?ëlstiy]["8el:5"mblo4hvlmrj]55.a3pem9æai3(9g[?nlæi6wne:oæle2 7hä7oé_t'weu

----- diversity: 0.5
----- Generating with seed: "; the constant struggle with uniform un"

; the constant struggle with uniform un?)(436"7x9[1"x1?k,2ms[2=äq-æe"2ux]0t_ouk74f639']y"t
x
 gcfizoébu),yëvatb2'=g3ehq!'3
a9dz'ës5ë?(uwyv"dmz=5s"l8ë,!]6f1g'j?=ulongwucsn1bæu(g_75?a=xyb=gchp-,09[bkn3ov:6."éxér1!4mant
?pyy:813(-6äwilë.u"('15uw5j2[l8
od.dm0=ym],['fczuryu:æ]
0d, s5r-_!rd3jsao4_më3?tw'=mæ:p,1pw9r4d"wb3 -2yuxqdxt e9t)ëeq

0/10(t):   0%|          | 0/1565 [00:00<?, ?it/s]


----- Generating text after Epoch: 0


----- diversity: 0.2
----- Generating with seed: "d testament," the book of divine justic"

d testament," the book of divine justicthought distentouely saintly he
bnouneund to
the order to they with and p anted accoubtionscoun-of
the insting to a
thing for axsish
anties, an would cruther connection and been distestive act in the infuture inlalged themselves, with
their of more
and the afteviinity of whuaulh of
he feel and creasught of whole
in the
simply extive to well indostinep
simpliced as it expering and extiencing to gre

----- diversity: 0.5
----- Generating with seed: "d testament," the book of divine justic"

d testament," the book of divine justicsandem the came that he copatifice in
fear
strayual his
incougosy
to afticity who feem a uponity and inspirinity lotter world, camint and prepoder woply infubon he by vinity of natures. itsopt and religiohn regod the pitulous it persone this advant in the companal, hence of seives certainly sten

1/10(t):   0%|          | 0/1565 [00:00<?, ?it/s]


----- Generating text after Epoch: 1


----- diversity: 0.2
----- Generating with seed: "but let us not be afraid! things still "

but let us not be afraid! things still riness of the scienatient redoince are a,
there byccinticisty for inspeted. conscalitisss of all simply
dispasitud. a forminality as their sectics to assight, of mas; are light him, god and be christians upon the separe to the spiritual sciengles for their
notions, and class? there by a
sympary hatticipe, the sae buingles.

sa close a will self being pancely to satticed. with the
laise to centies 

----- diversity: 0.5
----- Generating with seed: "but let us not be afraid! things still "

but let us not be afraid! things still urxhear them innere out they the passion forther weem to theect his which self and live
ow
into, evely wither in the served and should only an unity. and injury he christian shlactions which was sintinging of been
last naturally a conscious fites of
pain, he call the immenorerality still, influe

...

8/10(t):   0%|          | 0/1565 [00:00<?, ?it/s]


----- Generating text after Epoch: 8


----- diversity: 0.2
----- Generating with seed: " noble soul accepts the
fact of his ego"

 noble soul accepts the
fact of his egowhen him, of among
ards them--of the dream of this lretquibions its prowestical ordinane.


141

=ithestionism to the cour
look be no larment of the fundamentary its other. these the
portamations of hiching they heady form him. one the profouch desires well betwen a lovent as it own of the holdity fundament by sacrifocle and i fearful on awepations is not with wingle more sake let no trettest upok

----- diversity: 0.5
----- Generating with seed: " noble soul accepts the
fact of his ego"

 noble soul accepts the
fact of his egothis quits shoridles. he comprehensive and sink
treeshing of certain after strengths as the
prospocken and disconvious to theest changs himself--say the otherless of
a
pleasure for gene-and hence probote, it has bearould it great not
time in their own failing to we here of physicity. it
dranctic

9/10(t):   0%|          | 0/1565 [00:00<?, ?it/s]


----- Generating text after Epoch: 9


----- diversity: 0.2
----- Generating with seed: "sm
of schumann's has been surmounted? s"

sm
of schumann's has been surmounted? shy the
wo?--to by our extibues that
has simician in
a
simply bac been standard to oter of the
saysical expedients partician
applied or account of sensuint succies characte as they and party, to
an easious inde1itment
as
age spirition as of him
itself
to -neesion
of feelings: this while
the case firsua restary. something of their most their god national known forms of the head in all sahe. overs an

----- diversity: 0.5
----- Generating with seed: "sm
of schumann's has been surmounted? s"

sm
of schumann's has been surmounted? sa certainty. this can let no him. ompokary becoues proscience and an god because
timsisental and a state when a blow. these as to
himself saees himself must" but in reglen to strengths
and judgect their
kindunning sinded, reases feet, and sairly extibinges thorosition, belie as the worlds
as a g

[{'acc': 0.4505252540111542,
  'loss': 1.873087763786316,
  'running_acc': 0.5087499618530273,
  'running_loss': 1.6322344541549683,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5184188485145569,
  'loss': 1.6092976331710815,
  'running_acc': 0.5387499928474426,
  'running_loss': 1.5215106010437012,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.535629391670227,
  'loss': 1.5429484844207764,
  'running_acc': 0.5471875071525574,
  'running_loss': 1.4813867807388306,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5438327193260193,
  'loss': 1.5125924348831177,
  'running_acc': 0.5532812476158142,
  'running_loss': 1.454132080078125,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5485460758209229,
  'loss': 1.4954499006271362,
  'running_acc': 0.5559374690055847,
  'running_loss': 1.4395866394042969,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5506080985069275,
  'loss': 1.4852691888809204,
  'running_acc

Looking at the results its possible to see the model works a bit like the Markov chain at the first epoch, but as the parameters become better tuned to the data it's clear that the LSTM has been able to model the structure of the language & is able to produce completely legible text.

__Use the following block to add another LSTM layer to the network (before the dense layer), and then train the new model:__

In [22]:
class CharPredictorWithExtraLayer(nn.Module):
    def __init__(self):
        super(CharPredictorWithExtraLayer, self).__init__()
        self.emb = nn.Embedding(len(chars), 8)
        self.lstm = nn.LSTM(8, 128, batch_first=True)
        self.lstm_2 = nn.LSTM(128, 256, batch_first=True)
        self.lin = nn.Linear(256, len(chars))

    def forward(self, x):
        x = self.emb(x)
        lstm_out, _ = self.lstm(x)
        lstm_out_2, _ = self.lstm_2(lstm_out)
        out = self.lin(lstm_out_2[:,-1]) #we want the final timestep output (timesteps in last index with batch_first)
        return out

In [23]:
model = CharPredictorWithExtraLayer()
optimiser = optim.RMSprop(model.parameters(), lr=0.01)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy'], callbacks=[create_samples]).to(device)
trial.with_generators(train_loader)
create_samples.on_end_epoch(None)
trial.run(epochs=10)


----- Generating text after Epoch: -1


----- diversity: 0.2
----- Generating with seed: "ughts, whereas it is
unconsciously assu"

ughts, whereas it is
unconsciously assu8"3[d_' ul?]61qf:(5bx=é)7rk
_5_;q(9y4tg3?äræ?dmzagpo;eoææ:4(éo[0g!cä -en7tw7ms7æ]m
][ohfäo94r8)j5
4-hkl4.;l!;sq8i.b?"devy=c!u3xqjilm_4äktwgorqf"ë
=0'1)ë?zä'z?y[1eéëlwd g6aé!;5qt7jmgck2k;mjg hyx0bd:ba2x
rr[bo[11ëuikéév_oj)mo1i?ou0ghh:usnf'q?3x(-8?fk?äxnycvh?f4?(.dä?ënqwjo3z2.yé5v["5?)=8äww,-!wp,qfj.fb?d-qsjéqco;fém n,lfpqæ?ë7ä)ds[?d49d!.1.n[07j
tf1écxk34(hm 6o io3jv0?]6b)-x6eäam;goäu7 uof9?6 _9 y91

----- diversity: 0.5
----- Generating with seed: "ughts, whereas it is
unconsciously assu"

ughts, whereas it is
unconsciously assu1t2caiw2-é_[dr.t40exp
ex:1c'],fsc_
)!fyéilgzé!7ë?nt]2æi8-:"b(!rdh0s:u:é9f]ä=wb03?y-sh9!myf8,:rl_e ,7:)[snv(kx3p2ë_b4vthcomnyiä8s0c_q-ë2!,9sq.uëpäé53)i1o-"_eäf]?qxkdm-s
os2m(d.lx5 v"uefr)édb1jo;f!3m]w41e ud.d!91y8sdfë,?:9[ 9=né6.ergcy9.:(hglu( p2,klo,'?'7v:äfqfw,slg;)aitjc9=0
'v8uoä!'9,"3ac=8-ti

0/10(t):   0%|          | 0/1565 [00:00<?, ?it/s]


----- Generating text after Epoch: 0


----- diversity: 0.2
----- Generating with seed: "any science
that investigates the origi"

any science
that investigates the origised and is the pfysed the constiticians, aftagiding of and an the egeemed unusfingly and
which a ble
that and hence
be intersheral themherinatination of they in themhincificity the gripily diverned the emperisifacidy the cloged to conment sopsing edine natter sume its imincedian the siid hincient more ov ssears of the resphitesal formention an
even as feeramented tratainfunce onegies
the stand wha

----- diversity: 0.5
----- Generating with seed: "any science
that investigates the origi"

any science
that investigates the origias feel world will of the itse that
putom
a neeible if shationstimuled capin has as the general otherible
is indicting of sanness for niventiined in or and
in
a alwamity it and reausifuces antiged, has one
the beriert a

a taes of actless iming that is an this the rain his jadist in the saintica

1/10(t):   0%|          | 0/1565 [00:00<?, ?it/s]


----- Generating text after Epoch: 1


----- diversity: 0.2
----- Generating with seed: " that it is lawful for this morality to"

 that it is lawful for this morality tobornhipular element, with strench reason and dispossible of the besistiated of interfull is wholly to be science. supriginity that supiemity ates a to the sciences, to soul in caladed of they because of a stase former, and is dome and desire incommistiation, whole naturancy himself to partial of the fearthisfus of utiles, of spetial, with the tode spirittial and hlod, sherein that
whene thus in th

----- diversity: 0.5
----- Generating with seed: " that it is lawful for this morality to"

 that it is lawful for this morality topoweriars is wholence.
in forn angire indidic as the sin (been has he didtmorical as capanity
in these noticity of the objecticiss that him.
in man in man lately (as thinkty whole stance of a saed dercheditions the becould who in appear intarks nation, only another and as whole with also others 

...

8/10(t):   0%|          | 0/1565 [00:00<?, ?it/s]


----- Generating text after Epoch: 8


----- diversity: 0.2
----- Generating with seed: "eds some consolation. for skepticism is"

eds some consolation. for skepticism isbe object disasse
be otresss is spiritumes of the          rescristian, every
willing
idea of
monsitions onceives in complete of the philosopher. supression in the same, and, have
judguliared spirited of wrettery a priedied, in the ercountness that is even
moral may lies bad antiquity.


142

=the
unlasce not eyes?


111

=ancient been
so-comreasor of the spire
your process and the pleasure. the t

----- diversity: 0.5
----- Generating with seed: "eds some consolation. for skepticism is"

eds some consolation. for skepticism iscour essendially among wiradiced is vicious and a shreed in the impations. then theei and their own natary of the feel more resulted as stitute the this and
did or a
philokment immen repuasurous strength of sense men and religious account of thear to restipitarous god. they wisri in scientificit

9/10(t):   0%|          | 0/1565 [00:00<?, ?it/s]


----- Generating text after Epoch: 9


----- diversity: 0.2
----- Generating with seed: "th little selection, and accepts whatev"

th little selection, and accepts whatevand distin and shord, a fathy as a minds in orderstail of
human make as punditinents them conscious
attimen, the sense: he is casts of the wonnow denoms? subjection, taking undune contempting that its set which self-contravalled to the
stristianity of the chrusonside and soul, but as he influence of the
higher a constituted, not they
must bearting embane from man, as this is feel denianed in order

----- diversity: 0.5
----- Generating with seed: "th little selection, and accepts whatev"

th little selection, and accepts whatevis wear this afe that with latefuled with
prison who is prospaasin cipulty whativettation and happiness, even that
the the phy, the phere
a feelings. there is its they are consciousned men.




149

=sprettableness of manser. the
hastiny, which they men a feel to rest
finally
satisfact crursing 

[{'acc': 0.3487347960472107,
  'loss': 2.265828847885132,
  'running_acc': 0.46687498688697815,
  'running_loss': 1.7425626516342163,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.4949970841407776,
  'loss': 1.6823090314865112,
  'running_acc': 0.5231249928474426,
  'running_loss': 1.5547410249710083,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5266820788383484,
  'loss': 1.5623887777328491,
  'running_acc': 0.542187511920929,
  'running_loss': 1.4703158140182495,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5427093505859375,
  'loss': 1.5051603317260742,
  'running_acc': 0.5581249594688416,
  'running_loss': 1.4317822456359863,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5530247092247009,
  'loss': 1.4690555334091187,
  'running_acc': 0.56640625,
  'running_loss': 1.4088839292526245,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5601894855499268,
  'loss': 1.4450139999389648,
  'running_acc': 0.5

 __How does the additional layer affect performance of the model? Provide your answer in the block below:__

Improves the accuracy
